<a href="https://colab.research.google.com/github/vamsikrishna00466/stock-market-anlalysis/blob/main/ST_v1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#change timeframe to 5minutes
new_df = Change_Timeframe(df,"5Min")
new_df["Date"] = list(map(lambda x: str(x.date()),new_df.DateTime))
new_df["Time"] = list(map(lambda x: x.time(),new_df.DateTime))


#change data type decimal to float
new_df['Open_15'] = new_df['Open_15'].astype('float')
new_df['High_15'] = new_df['High_15'].astype('float')
new_df['Low_15'] = new_df['Low_15'].astype('float')
new_df['Close_15'] = new_df['Close_15'].astype('float')

#new_df.to_excel("sql_cash_data_5min.xlsx")

new_df = TrueRange(new_df)
ATR_df = average_true_range(data = new_df, period=14, drop_tr=True, smoothing="SMA")
f=3
n=15


df = ATR_df.copy()
#Calculation of SuperTrend
df['Upper Basic']=(df['High_15']+df['Low_15'])/2+(f*df['atr_14_SMA'])
df['Lower Basic']=(df['High_15']+df['Low_15'])/2-(f*df['atr_14_SMA'])
df['Upper Band']=df['Upper Basic']
df['Lower Band']=df['Lower Basic']
for i in range(n,len(df)):
  if df['Close_15'][i-1]<=df['Upper Band'][i-1]:
    df['Upper Band'][i]=min(df['Upper Basic'][i],df['Upper Band'][i-1])
  else:
    df['Upper Band'][i]=df['Upper Basic'][i]    
for i in range(n,len(df)):
  if df['Close_15'][i-1]>=df['Lower Band'][i-1]:
    df['Lower Band'][i]=max(df['Lower Basic'][i],df['Lower Band'][i-1])
  else:
    df['Lower Band'][i]=df['Lower Basic'][i]   
    df['SuperTrend']=np.nan
for i in df['SuperTrend']:
  if df['Close_15'][n-1]<=df['Upper Band'][n-1]:
    df['SuperTrend'][n-1]=df['Upper Band'][n-1]
  elif df['Close_15'][n-1]>df['Upper Band'][i]:
    df['SuperTrend'][n-1]=df['Lower Band'][n-1]
for i in range(n,len(df)):
  if df['SuperTrend'][i-1]==df['Upper Band'][i-1] and df['Close_15'][i]<=df['Upper Band'][i]:
    df['SuperTrend'][i]=df['Upper Band'][i]
  elif  df['SuperTrend'][i-1]==df['Upper Band'][i-1] and df['Close_15'][i]>=df['Upper Band'][i]:
    df['SuperTrend'][i]=df['Lower Band'][i]
  elif df['SuperTrend'][i-1]==df['Lower Band'][i-1] and df['Close_15'][i]>=df['Lower Band'][i]:
    df['SuperTrend'][i]=df['Lower Band'][i]
  elif df['SuperTrend'][i-1]==df['Lower Band'][i-1] and df['Close_15'][i]<=df['Lower Band'][i]:
    df['SuperTrend'][i]=df['Upper Band'][i]
  

df = df[['DateTime', 'Date', 'Time', 'Open_15', 'High_15', 'Low_15', 'Close_15', 'SuperTrend']]
df2 =df.copy()

#donchan channel 50 candles period
k = 50
df2['upper_channel'] = np.nan
df2['lower_channel'] = np.nan
df2['middle_channel'] = np.nan

df2['upper_channel'] = df2['High_15'].rolling(k).max().shift(k-50)
df2['lower_channel'] = df2['Low_15'].rolling(k).min().shift(k-50)
df2['middle_channel'] = ((df2['upper_channel'] + df2['lower_channel'])/2)

#df2.to_excel("super_trend_v1_1.xlsx")




In [ ]:
df2 = df2.dropna(axis=0, inplace=False)
df2=df2.reset_index()

In [ ]:
Trade = pd.DataFrame(columns=["Date","Trade Type","Bought At","Sold At","BuyTime","SellTime","Exit Type","Profit/Loss"])
n=0
LongSale=False
ShortSale=False
StopLoss = 0
StopLosses = []
for i in range(n, len(df2)):
  if (LongSale==False) & (ShortSale==False) & (df2.loc[i,"Time"]>=datetime.datetime.strptime('10:00:00', '%H:%M:%S').time()) & (df2.loc[i,"Time"]<datetime.datetime.strptime('15:00:00', '%H:%M:%S').time()):
    if (df2['upper_channel'][i] <= df2['High_15'][i]):
      BuyAt = df2.loc[i,"upper_channel"]
      BuyTime = df2.loc[i,"Time"]
      date = df2.loc[i,"Date"]
      InitialStopLoss = StopLoss
      LongSale=True
      continue
    elif (df2['lower_channel'][i] >= df2['Low_15'][i]):
      SoldAt = df2.loc[i,"lower_channel"]
      SellTime = df2.loc[i,"Time"]
      date = df2.loc[i,"Date"]
      InitialStopLoss = StopLoss
      ShortSale=True
      continue
  #If Time==15:15:00, we would need to square off our trade:
  if (df2.loc[i,"Time"]>=datetime.datetime.strptime('15:15:00', '%H:%M:%S').time()):
    if (LongSale==True):
      SoldAt = df2.loc[i,"Close_15"]
      SellTime = df2.loc[i,"Time"]
      date = df2.loc[i,"Date"]
      LongSale = False
      StopLosses = []
      Trade = Trade.append({"Date":df2.loc[i,"Date"],
                            "Trade Type":"Long Sale",
                            "Bought At":BuyAt,
                            "Sold At":SoldAt,
                            "BuyTime":BuyTime,
                            "SellTime":SellTime,
                            "Exit Type":"15:15:00 Exit",
                            "Profit/Loss":(SoldAt-BuyAt)},
                           ignore_index=True)
      continue
    #If we have made a Short Sale, we will square it off:
    elif (ShortSale==True):
      BuyAt = df2.loc[i,"Close_15"]
      BuyTime = df2.loc[i,"Time"]
      date = df2.loc[i,"Date"]
      ShortSale = False
      StopLosses = []
      Trade = Trade.append({"Date":df2.loc[i,"Date"],
                            "Trade Type":"Short Sale",
                            "Bought At":BuyAt,
                            "Sold At":SoldAt,
                            "BuyTime":BuyTime,
                            "SellTime":SellTime,
                            "Exit Type":"15:15:00 Exit",
                            "Profit/Loss":(SoldAt-BuyAt)},
                           ignore_index=True)
      continue
  #If we have made a Long Sale, and the current closing price hits StopLoss, Square it off
  if (LongSale==True) & (df2.loc[i,"Low_15"]<=df2.loc[i,"SuperTrend"]):
    SoldAt = df2.loc[i,"SuperTrend"]
    SellTime = df2.loc[i,"Time"]
    date = df2.loc[i,"Date"]
    LongSale = False
    StopLosses = []
    Trade = Trade.append({"Date":df2.loc[i,"Date"],
                          "Trade Type":"Long Sale",
                          "Bought At":BuyAt,
                          "Sold At":SoldAt,
                          "BuyTime":BuyTime,
                          "SellTime":SellTime,
                          "Exit Type":"StopLoss Triggered",
                          "Profit/Loss":(SoldAt-BuyAt)},
                          ignore_index=True)
    continue

    #If we have made a Short Sale, and the current closing price hits StopLoss, Square it off
  elif (ShortSale==True) & (df2.loc[i,"High_15"]>=df2.loc[i,"SuperTrend"]):
      BuyAt = df2.loc[i,"SuperTrend"]
      BuyTime = df2.loc[i,"Time"]
      date = df2.loc[i,"Date"]
      ShortSale = False
      StopLosses = []
      Trade = Trade.append({"Date":df2.loc[i,"Date"],
                            "Trade Type":"Short Sale",
                            "Bought At":BuyAt,
                            "Sold At":SoldAt,
                            "BuyTime":BuyTime,
                            "SellTime":SellTime,
                            "Exit Type":"StopLoss Triggered",
                            "Profit/Loss":(SoldAt-BuyAt)},
                            ignore_index=True)
      
#Trade.to_excel("test_ works1'.xlsx")


In [ ]:
#Trade.to_excel("trade1.0'.xlsx")
#df2.to_excel("data.1.0'.xlsx")



In [ ]:
df = Trade.copy()

df=df[['Date' ,'Trade Type', 'Bought At', 'Sold At', 'BuyTime', 'SellTime',  'Profit/Loss']]

#add time
df.loc[df['Trade Type'] == 'Long Sale', 'Entry Date Time'] = df['Date'].astype(str)+' '+df['BuyTime'].astype(str)
df.loc[df['Trade Type'] == 'Short Sale', 'Entry Date Time'] = df['Date'].astype(str)+' '+df['SellTime'].astype(str)
df.loc[df['Trade Type'] == 'Long Sale', 'Ex. Date Time'] = df['Date'].astype(str)+' '+df['SellTime'].astype(str)
df.loc[df['Trade Type'] == 'Short Sale', 'Ex. Date Time'] = df['Date'].astype(str)+' '+df['BuyTime'].astype(str)

#entry exit
df.loc[df['Trade Type'] == 'Long Sale', 'EntryPrice'] = df['Bought At']
df.loc[df['Trade Type'] == 'Short Sale', 'EntryPrice'] = df['Sold At']
df.loc[df['Trade Type'] == 'Long Sale', 'Exit Price'] = df['Sold At']
df.loc[df['Trade Type'] == 'Short Sale', 'Exit Price'] = df['Bought At']


#change Trade type column name to trade
df.loc[df['Trade Type'] == 'Long Sale', 'Trade'] = 'Long'
df.loc[df['Trade Type'] == 'Short Sale', 'Trade'] = 'Short'
df=df[['Date', 'Trade', 'Entry Date Time', 'Ex. Date Time', 'EntryPrice', 'Exit Price', 'Profit/Loss']]
df["Year-Month"] = list(map(lambda x: x[:7],df["Date"]))

#save file to xlsx
#df.to_excel("sample_output.xlsx")



df['year'] = pd.DatetimeIndex(df['Year-Month']).year
df['month']=pd.DatetimeIndex(df['Year-Month']).month


mresult = pd.DataFrame(columns = ['year','month','Profit/Loss'])
i = 0
j = -1
size = df.shape[0]
year = (df['year'][i])
month = (df['month'][i])
while(i<size):
    total = 0
    while (i<size) and (df['month'][i]) == month:
        total = total + df['Profit/Loss'][i]
        i +=1
    j += 1
    mresult.loc[j] = [year,month,total]
    month= month +1
    if (month == 13):
        year = year + 1 
        month = 1



yresult = pd.DataFrame(columns = ['year','Profit/Loss'])
i = 0
j = -1
size = mresult.shape[0]
year = (mresult['year'][i])
while(i<size):
    total = 0
    while (i<size) and (mresult['year'][i]) == year:
        total = total + mresult['Profit/Loss'][i]
        i +=1
    j += 1
    yresult.loc[j] = [year,total]
    year = year + 1

In [ ]:
#yresult.to_excel("yresult.1.0'.xlsx")
#mresult.to_excel("mresult.1.0'.xlsx")
#df.to_excel("raw_result.1.0'.xlsx")

